# Purpose
The purpose of this notebook is to emulate the SMS (semi-local momentum space) chiral potential using the NVP and KVP emulators. The potential has units of fm.

# Notebook Setup

## Importing Python libraries

In [1]:
import numpy as np
from numpy.typing import ArrayLike
from typing import Union, Optional
from ruamel.yaml import YAML
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
import h5py

%matplotlib inline
%load_ext autoreload    
%autoreload 2

## Importing local libraries

In [2]:
from emulate_kvp import setup_rc_params, plot_results
from emulate_kvp import plot_cross_section, plot_spin_obs
from emulate_kvp import compute_errors, spin_obs_errors
from emulate_kvp.utils import compute_mixed_S

from emulate_kvp import LHS_setup
from emulate_kvp import KVP_emulator
from emulate_kvp import Observables
from emulate_kvp import EmulateCrossSection
from emulate_kvp import EmulateSpinObservables

from emulate_nvp import BoundaryCondition
from emulate_nvp import ScatteringSystem
from emulate_nvp import TwoBodyScattering as NVP_emulator
from emulate_nvp import ObservablesEmulator

# Emulator setup

## Parameters

In [3]:
### PLOTTING ###
setup_rc_params(dpi=800)

### LHS PARAMETERS ###
mult_factor = 4 # Used to basis size: mult_factor * len(lecs) per partial wave
num_test = 500 # Size of testing arrays for emulator
vary_params = [-5, 5] # Range used for sampling parameter sets

### EMULATOR PARAMETERS ###
inc_param = False # If True, includes testing parameter set in training portion
emu_method = 'all' # Boundary conditions for emulator: 'K', '1/K', 'T', 'S', 'all'
solver = 'lstsq' # How to solve for prediction: 'lstsq', 'solve', 'pinv'
nugget = 1e-10 # Nugget for emulator calculations
seed = 1 # Random number seed for reproducibility

## Unpack potentials from h5 file for training

In [4]:
file_name = '../data/SMS_n4lo+_Lambda450MeV_jmax-20_np_5_036_402020.h5'

with h5py.File(file_name, "r") as f:
    jmax = int(f['jmax'][...])
    chiral_order = int(f['chiral_order'][...])
    cutoff = int(f['cutoff'][...])
    interaction = str(f['interaction'][...])[2:-1]
    
    E = f['E'][...]
    k = f['k'][...]
    degrees = f['degrees'][...]
    ps = f['ps'][...]
    ws = f['ws'][...]
    mesh_nodes = f['mesh_nodes'][...]
    pts_per_region = f['pts_per_region'][...]
    
    V_1S0 = f['V_1S0'][...]
    V0_1S0 = f['V0_1S0'][...]
    V1_1S0 = f['V1_1S0'][...]
    V_3P0 = f['V_3P0'][...]
    V0_3P0 = f['V0_3P0'][...]
    V1_3P0 = f['V1_3P0'][...]
    V_1P1 = f['V_1P1'][...]
    V0_1P1 = f['V0_1P1'][...]
    V1_1P1 = f['V1_1P1'][...]
    V_3P1 = f['V_3P1'][...]
    V0_3P1 = f['V0_3P1'][...]
    V1_3P1 = f['V1_3P1'][...]
    V_1D2 = f['V_1D2'][...]
    V0_1D2 = f['V0_1D2'][...]
    V1_1D2 = f['V1_1D2'][...]
    V_3D2 = f['V_3D2'][...]
    V0_3D2 = f['V0_3D2'][...]
    V1_3D2 = f['V1_3D2'][...]
    V_1F3 = f['V_1F3'][...]
    V0_1F3 = f['V0_1F3'][...]
    V1_1F3 = f['V1_1F3'][...]
    V_3F3 = f['V_3F3'][...]
    V0_3F3 = f['V0_3F3'][...]
    V1_3F3 = f['V1_3F3'][...]
    V_1 = f['V_1'][...]
    V_2 = f['V_2'][...]
    V_3S1_3D1 = f['V_3S1_3D1'][...]
    V0_3S1_3D1 = f['V0_3S1_3D1'][...]
    V1_3S1_3D1 = f['V1_3S1_3D1'][...]
    V_3P2_3F2 = f['V_3P2_3F2'][...]
    V0_3P2_3F2 = f['V0_3P2_3F2'][...]
    V1_3P2_3F2 = f['V1_3P2_3F2'][...]
    V_3D3_3G3 = f['V_3D3_3G3'][...]
    V0_3D3_3G3 = f['V0_3D3_3G3'][...]
    V1_3D3_3G3 = f['V1_3D3_3G3'][...]
    V_3F4_3H4 = f['V_3F4_3H4'][...]
    V0_3F4_3H4 = f['V0_3F4_3H4'][...]
    V1_3F4_3H4 = f['V1_3F4_3H4'][...]
    V_no_contacts = f['V_no_contacts'][...]

In [5]:
V_no_contacts_reshape = np.reshape(V_no_contacts, 
                                  (int(V_no_contacts.shape[0] / 6), 
                                   6, 
                                   V_no_contacts.shape[2], 
                                   V_no_contacts.shape[3], 
                                   V_no_contacts.shape[4]))

In [6]:
### MESH PARAMETERS ###
ki = mesh_nodes[0] # Mesh initial point
cut_reg1 = mesh_nodes[1] # Mesh cutoff of first region
cut_reg2 = mesh_nodes[2] # Mesh cutoff of second region
kf = mesh_nodes[3] # Mesh cutoff

pts_reg1 = pts_per_region[0] # Total points in first region
pts_reg2 = pts_per_region[1] # Total points in second region
pts_reg3 = pts_per_region[2] # Total points in third region
Ntot = pts_reg1 + pts_reg2 + pts_reg3 # Total mesh size

## Get coupling constants from YAML file

In [7]:
if (cutoff == 1):
    value = 400
elif (cutoff == 2):
    value = 450
elif (cutoff == 3):
    value = 500
elif (cutoff == 4):
    value = 550
    
location = '../data/' + str(value) + 'MeV/'

yaml = YAML(typ="safe")
with open(location + 'sms_lecs_n4lop_lam' + str(value) + '.yaml', "r") as input_file:
    lecs_yaml = yaml.load(input_file)

In [8]:
lecs_1S0 = [val for key, val in lecs_yaml.items() if "1S0" in key]
lecs_3P0 = [val for key, val in lecs_yaml.items() if "3P0" in key]
lecs_1P1 = [val for key, val in lecs_yaml.items() if "1P1" in key]
lecs_3P1 = [val for key, val in lecs_yaml.items() if "3P1" in key]
lecs_1D2 = [val for key, val in lecs_yaml.items() if "1D2" in key]
lecs_3D2 = [val for key, val in lecs_yaml.items() if "3D2" in key]
lecs_1F3 = [val for key, val in lecs_yaml.items() if "1F3" in key]
lecs_3F3 = [val for key, val in lecs_yaml.items() if "3F3" in key]

lecs_3S1_3D1 = [val for key, val in lecs_yaml.items() if "3S1" in key or "3D1" in key]
lecs_3P2_3F2 = [val for key, val in lecs_yaml.items() if "3P2" in key or "3F2" in key]
lecs_3D3_3G3 = [val for key, val in lecs_yaml.items() if "3D3" in key or "3G3" in key]
lecs_3F4_3H4 = [val for key, val in lecs_yaml.items() if "3F4" in key or "3H4" in key]

# Emulator calculation

## Uncoupled channels

### Partial wave: 1S0

In [9]:
jmom, wave = 0, '1S0'
n_b_1S0 = mult_factor * len(lecs_1S0)

basis_1S0 = LHS_setup(lecs_1S0, vary_params, n_b_1S0, num_test, 
                      inc_param=inc_param, fix_seed=seed)[0]

emu_1S0 = KVP_emulator(k, ps, ws, V0_1S0, V1_1S0, wave, is_coupled=False)
emu_1S0.train(basis_1S0, glockle=True, method=emu_method)
emu_pred_glockle_1S0 = emu_1S0.prediction(lecs_1S0, glockle=True, 
                                          sol=solver, h=nugget)

emu_1S0.train(basis_1S0, glockle=False, method=emu_method)
emu_pred_std_1S0 = emu_1S0.prediction(lecs_1S0, glockle=False, 
                                      sol=solver, h=nugget)

OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.


In [10]:
scatt_1S0 = NVP_emulator(
    V0=2 / np.pi * V0_1S0,
    V1=2 / np.pi * V1_1S0,
    k=ps, dk=ws, t_lab=E,
    system=ScatteringSystem(interaction),
    boundary_condition=BoundaryCondition.STANDING,
    nugget=nugget,
)
scatt_1S0.fit(basis_1S0);

### Partial wave: 3P0

In [11]:
jmom, wave = 0, '3P0'
n_b_3P0 = mult_factor * len(lecs_3P0)

basis_3P0 = LHS_setup(lecs_3P0, vary_params, n_b_3P0, num_test, 
                      inc_param=inc_param, fix_seed=seed)[0]

emu_3P0 = KVP_emulator(k, ps, ws, V0_3P0, V1_3P0, wave, is_coupled=False)
emu_3P0.train(basis_3P0, glockle=True, method=emu_method)
emu_3P0.train(basis_3P0, glockle=False, method=emu_method)

In [12]:
scatt_3P0 = NVP_emulator(
    V0=2 / np.pi * V0_3P0,
    V1=2 / np.pi * V1_3P0,
    k=ps, dk=ws, t_lab=E,
    system=ScatteringSystem(interaction),
    boundary_condition=BoundaryCondition.STANDING,
    nugget=nugget,
)
scatt_3P0.fit(basis_3P0);

### Partial wave: 1P1

In [13]:
jmom, wave = 1, '1P1'
n_b_1P1 = mult_factor * len(lecs_1P1)

basis_1P1 = LHS_setup(lecs_1P1, vary_params, n_b_1P1, num_test, 
                      inc_param=inc_param, fix_seed=seed)[0]

emu_1P1 = KVP_emulator(k, ps, ws, V0_1P1, V1_1P1, wave, is_coupled=False)
emu_1P1.train(basis_1P1, glockle=True, method=emu_method)
emu_1P1.train(basis_1P1, glockle=False, method=emu_method)

In [14]:
scatt_1P1 = NVP_emulator(
    V0=2 / np.pi * V0_1P1,
    V1=2 / np.pi * V1_1P1,
    k=ps, dk=ws, t_lab=E,
    system=ScatteringSystem(interaction),
    boundary_condition=BoundaryCondition.STANDING,
    nugget=nugget,
)
scatt_1P1.fit(basis_1P1);

### Partial wave: 3P1

In [15]:
jmom, wave = 1, '3P1'
n_b_3P1 = mult_factor * len(lecs_3P1)

basis_3P1 = LHS_setup(lecs_3P1, vary_params, n_b_3P1, num_test, 
                      inc_param=inc_param, fix_seed=seed)[0]

emu_3P1 = KVP_emulator(k, ps, ws, V0_3P1, V1_3P1, wave, is_coupled=False)
emu_3P1.train(basis_3P1, glockle=True, method=emu_method)
emu_3P1.train(basis_3P1, glockle=False, method=emu_method)

In [16]:
scatt_3P1 = NVP_emulator(
    V0=2 / np.pi * V0_3P1,
    V1=2 / np.pi * V1_3P1,
    k=ps, dk=ws, t_lab=E,
    system=ScatteringSystem(interaction),
    boundary_condition=BoundaryCondition.STANDING,
    nugget=nugget,
)
scatt_3P1.fit(basis_3P1);

### Partial wave: 1D2

In [17]:
jmom, wave = 2, '1D2'
n_b_1D2 = mult_factor * len(lecs_1D2)

basis_1D2 = LHS_setup(lecs_1D2, vary_params, n_b_1D2, num_test, 
                      inc_param=inc_param, fix_seed=seed)[0]

emu_1D2 = KVP_emulator(k, ps, ws, V0_1D2, V1_1D2, wave, is_coupled=False)
emu_1D2.train(basis_1D2, glockle=True, method=emu_method)
emu_1D2.train(basis_1D2, glockle=False, method=emu_method)

In [18]:
scatt_1D2 = NVP_emulator(
    V0=2 / np.pi * V0_1D2,
    V1=2 / np.pi * V1_1D2,
    k=ps, dk=ws, t_lab=E,
    system=ScatteringSystem(interaction),
    boundary_condition=BoundaryCondition.STANDING,
    nugget=nugget,
)
scatt_1D2.fit(basis_1D2);

### Partial wave: 3D2

In [19]:
jmom, wave = 2, '3D2'
n_b_3D2 = mult_factor * len(lecs_3D2)

basis_3D2 = LHS_setup(lecs_3D2, vary_params, n_b_3D2, num_test, 
                      inc_param=inc_param, fix_seed=seed)[0]

emu_3D2 = KVP_emulator(k, ps, ws, V0_3D2, V1_3D2, wave, is_coupled=False)
emu_3D2.train(basis_3D2, glockle=True, method=emu_method)
emu_3D2.train(basis_3D2, glockle=False, method=emu_method)

In [20]:
scatt_3D2 = NVP_emulator(
    V0=2 / np.pi * V0_3D2,
    V1=2 / np.pi * V1_3D2,
    k=ps, dk=ws, t_lab=E,
    system=ScatteringSystem(interaction),
    boundary_condition=BoundaryCondition.STANDING,
    nugget=nugget,
)
scatt_3D2.fit(basis_3D2);

### Partial wave: 1F3

In [21]:
jmom, wave = 3, '1F3'
n_b_1F3 = mult_factor * len(lecs_1F3)

basis_1F3 = LHS_setup(lecs_1F3, vary_params, n_b_1F3, num_test, 
                      inc_param=inc_param, fix_seed=seed)[0]

emu_1F3 = KVP_emulator(k, ps, ws, V0_1F3, V1_1F3, wave, is_coupled=False)
emu_1F3.train(basis_1F3, glockle=True, method=emu_method)
emu_1F3.train(basis_1F3, glockle=False, method=emu_method)

In [22]:
scatt_1F3 = NVP_emulator(
    V0=2 / np.pi * V0_1F3,
    V1=2 / np.pi * V1_1F3,
    k=ps, dk=ws, t_lab=E,
    system=ScatteringSystem(interaction),
    boundary_condition=BoundaryCondition.STANDING,
    nugget=nugget,
)
scatt_1F3.fit(basis_1F3);

### Partial wave: 3F3

In [23]:
jmom, wave = 3, '3F3'
n_b_3F3 = mult_factor * len(lecs_3F3)

basis_3F3 = LHS_setup(lecs_3F3, vary_params, n_b_3F3, num_test, 
                      inc_param=inc_param, fix_seed=seed)[0]

emu_3F3 = KVP_emulator(k, ps, ws, V0_3F3, V1_3F3, wave, is_coupled=False)
emu_3F3.train(basis_3F3, glockle=True, method=emu_method)
emu_3F3.train(basis_3F3, glockle=False, method=emu_method)

In [24]:
scatt_3F3 = NVP_emulator(
    V0=2 / np.pi * V0_3F3,
    V1=2 / np.pi * V1_3F3,
    k=ps, dk=ws, t_lab=E,
    system=ScatteringSystem(interaction),
    boundary_condition=BoundaryCondition.STANDING,
    nugget=nugget,
)
scatt_3F3.fit(basis_3F3);

## Coupled channels

### Partial wave: 3S1/3D1

In [25]:
jmom, wave = 1, '3S1/3D1'
n_b_3S1_3D1 = mult_factor * len(lecs_3S1_3D1)

basis_3S1_3D1 = LHS_setup(lecs_3S1_3D1, vary_params, n_b_3S1_3D1, num_test, 
                          inc_param=inc_param, fix_seed=seed)[0]

emu_3S1_3D1 = KVP_emulator(k, ps, ws, V0_3S1_3D1, V1_3S1_3D1, wave, is_coupled=True)
emu_3S1_3D1.train(basis_3S1_3D1, glockle=True, method=emu_method)
emu_3S1_3D1.train(basis_3S1_3D1, glockle=False, method=emu_method)

In [26]:
scatt_3S1_3D1 = NVP_emulator(
    V0=2 / np.pi * V0_3S1_3D1,
    V1=2 / np.pi * V1_3S1_3D1,
    k=ps, dk=ws, t_lab=E,
    system=ScatteringSystem(interaction),
    boundary_condition=BoundaryCondition.STANDING,
    is_coupled=True,
    nugget=nugget,
)
scatt_3S1_3D1.fit(basis_3S1_3D1);

### Partial wave: 3P2/3F2

In [27]:
jmom, wave = 2, '3P2/3F2'
n_b_3P2_3F2 = mult_factor * len(lecs_3P2_3F2)

basis_3P2_3F2 = LHS_setup(lecs_3P2_3F2, vary_params, n_b_3P2_3F2, num_test, 
                          inc_param=inc_param, fix_seed=seed)[0]

emu_3P2_3F2 = KVP_emulator(k, ps, ws, V0_3P2_3F2, V1_3P2_3F2, wave, is_coupled=True)
emu_3P2_3F2.train(basis_3P2_3F2, glockle=True, method=emu_method)
emu_3P2_3F2.train(basis_3P2_3F2, glockle=False, method=emu_method)

In [28]:
scatt_3P2_3F2 = NVP_emulator(
    V0=2 / np.pi * V0_3P2_3F2,
    V1=2 / np.pi * V1_3P2_3F2,
    k=ps, dk=ws, t_lab=E,
    system=ScatteringSystem(interaction),
    boundary_condition=BoundaryCondition.STANDING,
    is_coupled=True,
    nugget=nugget,
)
scatt_3P2_3F2.fit(basis_3P2_3F2);

### Partial wave: 3D3/3G3

In [29]:
jmom, wave = 3, '3D3/3G3'
n_b_3D3_3G3 = mult_factor * len(lecs_3D3_3G3)

basis_3D3_3G3 = LHS_setup(lecs_3D3_3G3, vary_params, n_b_3D3_3G3, num_test, 
                          inc_param=inc_param, fix_seed=seed)[0]

emu_3D3_3G3 = KVP_emulator(k, ps, ws, V0_3D3_3G3, V1_3D3_3G3, wave, is_coupled=True)
emu_3D3_3G3.train(basis_3D3_3G3, glockle=True, method=emu_method)
emu_3D3_3G3.train(basis_3D3_3G3, glockle=False, method=emu_method)

In [30]:
scatt_3D3_3G3 = NVP_emulator(
    V0=2 / np.pi * V0_3D3_3G3,
    V1=2 / np.pi * V1_3D3_3G3,
    k=ps, dk=ws, t_lab=E,
    system=ScatteringSystem(interaction),
    boundary_condition=BoundaryCondition.STANDING,
    is_coupled=True,
    nugget=nugget,
)
scatt_3D3_3G3.fit(basis_3D3_3G3);

### Partial wave: 3F4/3H4

In [31]:
jmom, wave = 4, '3F4/3H4'
n_b_3F4_3H4 = mult_factor * len(lecs_3F4_3H4)

basis_3F4_3H4 = LHS_setup(lecs_3F4_3H4, vary_params, n_b_3F4_3H4, num_test, 
                          inc_param=inc_param, fix_seed=seed)[0]

emu_3F4_3H4 = KVP_emulator(k, ps, ws, V0_3F4_3H4, V1_3F4_3H4, wave, is_coupled=True)
emu_3F4_3H4.train(basis_3F4_3H4, glockle=True, method=emu_method)
emu_3F4_3H4.train(basis_3F4_3H4, glockle=False, method=emu_method)

In [32]:
scatt_3F4_3H4 = NVP_emulator(
    V0=2 / np.pi * V0_3F4_3H4,
    V1=2 / np.pi * V1_3F4_3H4,
    k=ps, dk=ws, t_lab=E,
    system=ScatteringSystem(interaction),
    boundary_condition=BoundaryCondition.STANDING,
    is_coupled=True,
    nugget=nugget,
)
scatt_3F4_3H4.fit(basis_3F4_3H4);

## Emulating the total cross section

### Getting KVP emulators

In [33]:
jmom, wave = 4, None

emu_1 = KVP_emulator(k, ps, ws, V_1, 0, wave, is_coupled=False)
emu_2 = KVP_emulator(k, ps, ws, V_2, 0, wave, is_coupled=False)

In [34]:
inputs_uncoupled = [
    [0, lecs_1S0, V_1S0, emu_1S0],
    [0, lecs_3P0, V_3P0, emu_3P0],
    [1, lecs_1P1, V_1P1, emu_1P1],
    [1, lecs_3P1, V_3P1, emu_3P1],
    [2, lecs_1D2, V_1D2, emu_1D2],
    [2, lecs_3D2, V_3D2, emu_3D2],
    [3, lecs_1F3, V_1F3, emu_1F3],
    [3, lecs_3F3, V_3F3, emu_3F3],
    [4, None, V_1, emu_1],
    [4, None, V_2, emu_2],
]

inputs_coupled = [
    [1, lecs_3S1_3D1, V_3S1_3D1, emu_3S1_3D1],
    [2, lecs_3P2_3F2, V_3P2_3F2, emu_3P2_3F2],
    [3, lecs_3D3_3G3, V_3D3_3G3, emu_3D3_3G3],
    [4, lecs_3F4_3H4, V_3F4_3H4, emu_3F4_3H4],
]

inputs_no_contacts = [
    [j + 5, 
     None, 
     V_no_contacts_reshape[j][idx], 
     KVP_emulator(k, ps, ws, V_no_contacts_reshape[j][idx], 0, None)
    ] 
    for j in range(jmax - 4) for idx in range(6)
]

inputs_all = inputs_uncoupled + inputs_coupled + inputs_no_contacts
js, lecs, potentials, emulators = list(zip(*inputs_all))

### Getting NVP emulators

In [35]:
scatt_1 = NVP_emulator(
    V0=2 / np.pi * V_1,
    V1=2 / np.pi * np.zeros_like(V_1[..., None]),
    k=ps, dk=ws, t_lab=E,
    system=ScatteringSystem(interaction),
    boundary_condition=BoundaryCondition.STANDING,
    nugget=nugget,
)

scatt_2 = NVP_emulator(
    V0=2 / np.pi * V_2,
    V1=2 / np.pi * np.zeros_like(V_2[..., None]),
    k=ps, dk=ws, t_lab=E,
    system=ScatteringSystem(interaction),
    boundary_condition=BoundaryCondition.STANDING,
    nugget=nugget,
)

In [36]:
inputs_uncoupled_nvp = [
    [0, lecs_1S0, scatt_1S0],
    [0, lecs_3P0, scatt_3P0],
    [1, lecs_1P1, scatt_1P1],
    [1, lecs_3P1, scatt_3P1],
    [2, lecs_1D2, scatt_1D2],
    [2, lecs_3D2, scatt_3D2],
    [3, lecs_1F3, scatt_1F3],
    [3, lecs_3F3, scatt_3F3],
    [4, None, scatt_1],
    [4, None, scatt_2],
]

inputs_coupled_nvp = [
    [1, lecs_3S1_3D1, scatt_3S1_3D1],
    [2, lecs_3P2_3F2, scatt_3P2_3F2],
    [3, lecs_3D3_3G3, scatt_3D3_3G3],
    [4, lecs_3F4_3H4, scatt_3F4_3H4],
]

inputs_no_contacts_nvp = [
    [j + 5, 
     None,  
     NVP_emulator(
        V0=2 / np.pi * V_no_contacts_reshape[j][idx],
        V1=2 / np.pi * np.zeros_like(V_no_contacts_reshape[j][idx][..., None]),
        k=ps, dk=ws, t_lab=E,
        system=ScatteringSystem(interaction),
        boundary_condition=BoundaryCondition.STANDING,
        nugget=nugget,
    )
    ] 
    for j in range(jmax - 4) for idx in range(6)
]

inputs_all_nvp = inputs_uncoupled_nvp + inputs_coupled_nvp + inputs_no_contacts_nvp

j_nvp, lecs_nvp, emulators_nvp = list(zip(*inputs_all_nvp))

### Getting LECs

In [37]:
true_lecs = lecs_1S0 +  lecs_3P0 + lecs_1P1 + \
            lecs_3P1 + lecs_1D2 + lecs_3D2 + \
            lecs_1F3 + lecs_3F3 + lecs_3S1_3D1 + \
            lecs_3P2_3F2 + lecs_3D3_3G3 + lecs_3F4_3H4

lec_test_all = LHS_setup(true_lecs, vary_params, 0, num_test, fix_seed=seed)[1]
true_lecs_arr = np.array(true_lecs)
print(lec_test_all.shape)

(500, 25)


## Emulate spin observables

In [38]:
compute_obs = Observables(E, k, degrees)
spin_obs = EmulateSpinObservables(js, lecs, emulators, solver)
nvp_obs = ObservablesEmulator(emulators_nvp, lecs_nvp, 
                              j_nvp, compute_obs)

### Spin observables - Simulator

In [39]:
%%time
sp_obs_sim = spin_obs.emulate_spin_obs(compute_obs, true_lecs, 
                                       potentials, nugget, 
                                       glockle=None, emulate=False)

CPU times: user 45.7 s, sys: 1.34 s, total: 47.1 s
Wall time: 6.53 s


### Spin observables - KVP with Std

In [40]:
%%time
sp_obs_emu_std = spin_obs.emulate_spin_obs(compute_obs, true_lecs, 
                                           potentials, nugget, 
                                           glockle=False, emulate=True)

CPU times: user 4.6 s, sys: 481 ms, total: 5.09 s
Wall time: 1.7 s


In [41]:
dsg_sim = sp_obs_sim['DSG']
D_sim = sp_obs_sim['D']
Ay_sim = sp_obs_sim['PB']
Axx_sim = sp_obs_sim['AXX']
Ayy_sim = sp_obs_sim['AYY']
A_sim = sp_obs_sim['A']

dsg_emu_std = sp_obs_emu_std['DSG']
D_emu_std = sp_obs_emu_std['D']
Ay_emu_std = sp_obs_emu_std['PB']
Axx_emu_std = sp_obs_emu_std['AXX']
Ayy_emu_std = sp_obs_emu_std['AYY']
A_emu_std = sp_obs_emu_std['A']

In [42]:
dsg_rel_err_std = compute_errors(sp_obs_sim, sp_obs_emu_std, 'DSG')[1]
D_rel_err_std = compute_errors(sp_obs_sim, sp_obs_emu_std, 'D')[1]
Ay_rel_err_std = compute_errors(sp_obs_sim, sp_obs_emu_std, 'PB')[1]
Axx_rel_err_std = compute_errors(sp_obs_sim, sp_obs_emu_std, 'AXX')[1]
Ayy_rel_err_std = compute_errors(sp_obs_sim, sp_obs_emu_std, 'AYY')[1]
A_rel_err_std = compute_errors(sp_obs_sim, sp_obs_emu_std, 'A')[1]

### Spin observables - KVP with Glockle

In [43]:
%%time
sp_obs_emu_glockle = spin_obs.emulate_spin_obs(compute_obs, true_lecs, 
                                               potentials, nugget, 
                                               glockle=None, emulate=True)

CPU times: user 4.52 s, sys: 109 ms, total: 4.63 s
Wall time: 1.25 s


In [44]:
dsg_rel_err_glockle = compute_errors(sp_obs_sim, sp_obs_emu_glockle, 'DSG')[1]
D_rel_err_glockle = compute_errors(sp_obs_sim, sp_obs_emu_glockle, 'D')[1]
Ay_rel_err_glockle = compute_errors(sp_obs_sim, sp_obs_emu_glockle, 'PB')[1]
Ayy_rel_err_glockle = compute_errors(sp_obs_sim, sp_obs_emu_glockle, 'AYY')[1]
A_rel_err_glockle = compute_errors(sp_obs_sim, sp_obs_emu_glockle, 'A')[1]

### Spin observables - NVP

In [45]:
%%time
sp_obs_sim_nvp = nvp_obs.predict(true_lecs_arr, spin_obs=True, full_space=True)

CPU times: user 1min 12s, sys: 4.24 s, total: 1min 16s
Wall time: 11.1 s


In [46]:
%%time
sp_obs_emu_nvp = nvp_obs.predict(true_lecs_arr, spin_obs=True, full_space=False)

CPU times: user 2.09 s, sys: 143 ms, total: 2.23 s
Wall time: 692 ms


In [47]:
dsg_rel_err_nvp = compute_errors(sp_obs_sim_nvp, sp_obs_emu_nvp, 'DSG')[1]
D_rel_err_nvp = compute_errors(sp_obs_sim_nvp, sp_obs_emu_nvp, 'D')[1]
Ay_rel_err_nvp = compute_errors(sp_obs_sim_nvp, sp_obs_emu_nvp, 'PB')[1]
Axx_rel_err_nvp = compute_errors(sp_obs_sim_nvp, sp_obs_emu_nvp, 'AXX')[1]
Ayy_rel_err_nvp = compute_errors(sp_obs_sim_nvp, sp_obs_emu_nvp, 'AYY')[1]
A_rel_err_nvp = compute_errors(sp_obs_sim_nvp, sp_obs_emu_nvp, 'A')[1]

## Spin observables: predict with sampled parameter sets

In [48]:
spin_obs_sim_sample = np.zeros((num_test), dtype=object)
spin_obs_emu_std_sample = np.zeros((num_test), dtype=object)
spin_obs_emu_glockle_sample = np.zeros((num_test), dtype=object)
spin_obs_sim_nvp_sample = np.zeros((num_test), dtype=object)
spin_obs_emu_nvp_sample = np.zeros((num_test), dtype=object)

### Sampled spin observables - KVP emulator with Std

In [49]:
test_so_emu_std = %timeit -c -o -n 1 -r 1 spin_obs.predict(spin_obs_emu_std_sample, compute_obs, lec_test_all, potentials, nugget, glockle=False, emulate=True)


38min 17s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


### Sampled spin observables - KVP emulator with Glockle

In [50]:
test_so_emu_glockle = %timeit -c -o -n 1 -r 1 spin_obs.predict(spin_obs_emu_glockle_sample, compute_obs, lec_test_all, potentials, nugget, glockle=True, emulate=True)


38min 6s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


### Sampled spin observables - Simulator

In [51]:
test_so_sim = %timeit -c -o -n 1 -r 1 spin_obs.predict(spin_obs_sim_sample, compute_obs, lec_test_all, potentials, nugget, glockle=None, emulate=False)


5h 46min 59s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


### Sampled spin observables - NVP

In [52]:
test_so_emu_nvp = %timeit -c -o -n 1 -r 1 nvp_obs.predict(lec_test_all, spin_obs=True, full_space=False, out=spin_obs_emu_nvp_sample)

17min 11s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [53]:
test_so_sim_nvp = %timeit -c -o -n 1 -r 1 nvp_obs.predict(lec_test_all, spin_obs=True, full_space=True, out=spin_obs_sim_nvp_sample)

1h 43min 26s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


### Calculate errors

In [54]:
spin_obs_err_std = spin_obs_errors(E, degrees, 
                                   spin_obs_sim_sample, 
                                   spin_obs_emu_std_sample)

dsg_err = spin_obs_err_std[0]
D_err = spin_obs_err_std[1]
Ay_err = spin_obs_err_std[2]
Axx_err = spin_obs_err_std[3]
Ayy_err = spin_obs_err_std[4]
A_err = spin_obs_err_std[5]

dsg_mean_std = np.mean(dsg_err, axis=0)
D_mean_std = np.mean(D_err, axis=0)
Ay_mean_std = np.mean(Ay_err, axis=0)
Axx_mean_std = np.mean(Axx_err, axis=0)
Ayy_mean_std = np.mean(Ayy_err, axis=0)
A_mean_std = np.mean(A_err, axis=0)

dsg_std_angle_avg = np.mean(dsg_mean_std, axis=1)
D_std_angle_avg = np.mean(D_mean_std, axis=1)
Ay_std_angle_avg = np.mean(Ay_mean_std, axis=1)
Axx_std_angle_avg = np.mean(Axx_mean_std, axis=1)
Ayy_std_angle_avg = np.mean(Ayy_mean_std, axis=1)
A_std_angle_avg = np.mean(A_mean_std, axis=1)

In [55]:
spin_obs_err_glockle = spin_obs_errors(E, degrees, 
                                       spin_obs_sim_sample, 
                                       spin_obs_emu_glockle_sample)

dsg_glockle_err = spin_obs_err_glockle[0]
D_glockle_err = spin_obs_err_glockle[1]
Ay_glockle_err = spin_obs_err_glockle[2]
Axx_glockle_err = spin_obs_err_glockle[3]
Ayy_glockle_err = spin_obs_err_glockle[4]
A_glockle_err = spin_obs_err_glockle[5]

dsg_mean_glockle = np.mean(dsg_glockle_err, axis=0)
D_mean_glockle = np.mean(D_glockle_err, axis=0)
Ay_mean_glockle = np.mean(Ay_glockle_err, axis=0)
Axx_mean_glockle = np.mean(Axx_glockle_err, axis=0)
Ayy_mean_glockle = np.mean(Ayy_glockle_err, axis=0)
A_mean_glockle = np.mean(A_glockle_err, axis=0)

dsg_glockle_angle_avg = np.mean(dsg_mean_glockle, axis=1)
D_glockle_angle_avg = np.mean(D_mean_glockle, axis=1)
Ay_glockle_angle_avg = np.mean(Ay_mean_glockle, axis=1)
Axx_glockle_angle_avg = np.mean(Axx_mean_glockle, axis=1)
Ayy_glockle_angle_avg = np.mean(Ayy_mean_glockle, axis=1)
A_glockle_angle_avg = np.mean(A_mean_glockle, axis=1)

In [56]:
spin_obs_err_nvp = spin_obs_errors(E, degrees, 
                                   spin_obs_sim_nvp_sample, 
                                   spin_obs_emu_nvp_sample)

dsg_nvp_err = spin_obs_err_nvp[0]
D_nvp_err = spin_obs_err_nvp[1]
Ay_nvp_err = spin_obs_err_nvp[2]
Axx_nvp_err = spin_obs_err_nvp[3]
Ayy_nvp_err = spin_obs_err_nvp[4]
A_nvp_err = spin_obs_err_nvp[5]

dsg_mean_nvp = np.mean(dsg_nvp_err, axis=0)
D_mean_nvp = np.mean(D_nvp_err, axis=0)
Ay_mean_nvp = np.mean(Ay_nvp_err, axis=0)
Axx_mean_nvp = np.mean(Axx_nvp_err, axis=0)
Ayy_mean_nvp = np.mean(Ayy_nvp_err, axis=0)
A_mean_nvp = np.mean(A_nvp_err, axis=0)

dsg_nvp_angle_avg = np.mean(dsg_mean_nvp, axis=1)
D_nvp_angle_avg = np.mean(D_mean_nvp, axis=1)
Ay_nvp_angle_avg = np.mean(Ay_mean_nvp, axis=1)
Axx_nvp_angle_avg = np.mean(Axx_mean_nvp, axis=1)
Ayy_nvp_angle_avg = np.mean(Ayy_mean_nvp, axis=1)
A_nvp_angle_avg = np.mean(A_mean_nvp, axis=1)

### Sampled spin observables - Timing

In [57]:
test_so_sim.average / test_so_emu_std.average

9.062567896241873

In [58]:
test_so_sim.average / test_so_emu_glockle.average

9.106366294763466

In [59]:
test_so_sim_nvp.average / test_so_emu_nvp.average

6.016035311383513

### Sampled spin observables - Analytics

In [60]:
E1 = 5
E2 = 100
E3 = 200
E4 = 300
E_table = [E1, E2, E3, E4]

print('Avg. Rel. error: angle averaged')
print('--------------------------------------')

for E_i in E_table:
    print('Energy:', E_i)
    print('Obs. ', '  Std. ', ' Glockle ', ' NVP ')
    print(f'DSG: ' f'{dsg_std_angle_avg[E_i]:.2e} '
                   f'{dsg_glockle_angle_avg[E_i]:.2e} '
                   f'{dsg_nvp_angle_avg[E_i]:.2e}')
    print(f'D  : ' f'{D_std_angle_avg[E_i]:.2e} '
                   f'{D_glockle_angle_avg[E_i]:.2e} '
                   f'{D_nvp_angle_avg[E_i]:.2e}')
    print(f'Ay : ' f'{Ay_std_angle_avg[E_i]:.2e} '
                   f'{Ay_glockle_angle_avg[E_i]:.2e} '
                   f'{Ay_nvp_angle_avg[E_i]:.2e}')
    print(f'Axx: ' f'{Axx_std_angle_avg[E_i]:.2e} '
                   f'{Axx_glockle_angle_avg[E_i]:.2e} '
                   f'{Axx_nvp_angle_avg[E_i]:.2e}')
    print(f'Ayy: ' f'{Ayy_std_angle_avg[E_i]:.2e} '
                   f'{Ayy_glockle_angle_avg[E_i]:.2e} '
                   f'{Ayy_nvp_angle_avg[E_i]:.2e}')
    print(f'A  : ' f'{A_std_angle_avg[E_i]:.2e} '
                   f'{A_glockle_angle_avg[E_i]:.2e} '
                   f'{A_nvp_angle_avg[E_i]:.2e}')
    print('\n')

Avg. Rel. error: angle averaged
--------------------------------------
Energy: 5
Obs.    Std.   Glockle   NVP 
DSG: 8.81e-11 5.50e-08 2.94e-12
D  : 1.45e-09 3.94e-07 4.68e-11
Ay : 1.59e-09 8.81e-07 4.09e-11
Axx: 2.63e-09 4.09e-07 7.61e-11
Ayy: 3.01e-09 4.36e-07 7.73e-11
A  : 5.47e-09 7.97e-07 8.08e-11


Energy: 100
Obs.    Std.   Glockle   NVP 
DSG: 2.44e-13 3.10e-07 5.89e-14
D  : 2.41e-12 4.77e-06 5.11e-13
Ay : 7.84e-12 8.68e-06 1.96e-12
Axx: 7.41e-12 1.19e-05 2.05e-12
Ayy: 1.09e-11 9.69e-06 4.14e-12
A  : 4.01e-12 3.74e-06 1.52e-12


Energy: 200
Obs.    Std.   Glockle   NVP 
DSG: 9.24e-11 3.96e-05 1.33e-10
D  : 5.23e-10 2.60e-04 7.34e-10
Ay : 3.58e-09 9.90e-04 6.82e-09
Axx: 7.55e-10 4.09e-04 1.56e-09
Ayy: 1.57e-09 5.58e-04 2.02e-09
A  : 1.59e-09 5.51e-04 6.47e-09


Energy: 300
Obs.    Std.   Glockle   NVP 
DSG: 1.83e-12 8.86e-06 6.87e-13
D  : 2.89e-11 1.01e-04 3.16e-12
Ay : 3.68e-11 8.92e-05 1.59e-11
Axx: 2.60e-11 1.59e-04 8.34e-12
Ayy: 4.17e-11 1.63e-04 7.26e-12
A  : 2.36e-11 9.23e-0

In [61]:
E1 = 5
E2 = 100
E3 = 200
E4 = 300
E_table = [E1, E2, E3, E4]

print('Log of Avg. Rel. error: angle averaged')
print('--------------------------------------')

for E_i in E_table:
    print('Energy:', E_i)
    print('Obs. ', '  Std. ', ' Glockle ', ' NVP ')
    print(f'DSG: ' f'{np.log10(dsg_std_angle_avg[E_i]):.2e} '
                   f'{np.log10(dsg_glockle_angle_avg[E_i]):.2e} '
                   f'{np.log10(dsg_nvp_angle_avg[E_i]):.2e}')
    print(f'D  : ' f'{np.log10(D_std_angle_avg[E_i]):.2e} '
                   f'{np.log10(D_glockle_angle_avg[E_i]):.2e} '
                   f'{np.log10(D_nvp_angle_avg[E_i]):.2e}')
    print(f'Ay : ' f'{np.log10(Ay_std_angle_avg[E_i]):.2e} '
                   f'{np.log10(Ay_glockle_angle_avg[E_i]):.2e} '
                   f'{np.log10(Ay_nvp_angle_avg[E_i]):.2e}')
    print(f'Axx: ' f'{np.log10(Axx_std_angle_avg[E_i]):.2e} '
                   f'{np.log10(Axx_glockle_angle_avg[E_i]):.2e} '
                   f'{np.log10(Axx_nvp_angle_avg[E_i]):.2e}')
    print(f'Ayy: ' f'{np.log10(Ayy_std_angle_avg[E_i]):.2e} '
                   f'{np.log10(Ayy_glockle_angle_avg[E_i]):.2e} '
                   f'{np.log10(Ayy_nvp_angle_avg[E_i]):.2e}')
    print(f'A  : ' f'{np.log10(A_std_angle_avg[E_i]):.2e} '
                   f'{np.log10(A_glockle_angle_avg[E_i]):.2e} '
                   f'{np.log10(A_nvp_angle_avg[E_i]):.2e}')
    print('\n')

Log of Avg. Rel. error: angle averaged
--------------------------------------
Energy: 5
Obs.    Std.   Glockle   NVP 
DSG: -1.01e+01 -7.26e+00 -1.15e+01
D  : -8.84e+00 -6.40e+00 -1.03e+01
Ay : -8.80e+00 -6.05e+00 -1.04e+01
Axx: -8.58e+00 -6.39e+00 -1.01e+01
Ayy: -8.52e+00 -6.36e+00 -1.01e+01
A  : -8.26e+00 -6.10e+00 -1.01e+01


Energy: 100
Obs.    Std.   Glockle   NVP 
DSG: -1.26e+01 -6.51e+00 -1.32e+01
D  : -1.16e+01 -5.32e+00 -1.23e+01
Ay : -1.11e+01 -5.06e+00 -1.17e+01
Axx: -1.11e+01 -4.92e+00 -1.17e+01
Ayy: -1.10e+01 -5.01e+00 -1.14e+01
A  : -1.14e+01 -5.43e+00 -1.18e+01


Energy: 200
Obs.    Std.   Glockle   NVP 
DSG: -1.00e+01 -4.40e+00 -9.88e+00
D  : -9.28e+00 -3.58e+00 -9.13e+00
Ay : -8.45e+00 -3.00e+00 -8.17e+00
Axx: -9.12e+00 -3.39e+00 -8.81e+00
Ayy: -8.81e+00 -3.25e+00 -8.69e+00
A  : -8.80e+00 -3.26e+00 -8.19e+00


Energy: 300
Obs.    Std.   Glockle   NVP 
DSG: -1.17e+01 -5.05e+00 -1.22e+01
D  : -1.05e+01 -4.00e+00 -1.15e+01
Ay : -1.04e+01 -4.05e+00 -1.08e+01
Axx: -1.06e+01 